# Modeling-baseline


經過初步對國道五號北向的平均旅行時間的探索  
我們已知在不同時間段、不同門架間的時間序列表現不同  
如果能有效運用多個特徵去對時間序列進行預測會是不錯的成果  
因此在完成我們的模型之前，我們需要先建構baseline，作為後續比較的基礎  

我們會使用NIXTLA開發的statsforecast套件來做baseline  
主要流程如下:
* 使用最基本的時間序列模型進行建模，採用多個指標來觀測誤差落在哪裡  
* 透過cross_validation來評估建模的狀況如何  

# Lib

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import time
import sys

# 將 hwttp 資料夾加入 Python 搜尋路徑
sys.path.append(os.path.abspath('..'))
import hwttp.hwtoolkit as tk

# traditional method as baseline
from statsforecast import StatsForecast
from statsforecast.models import(
    AutoARIMA,
    SeasonalNaive,
    MSTL
)

# evaluation
from utilsforecast.losses import mse, mae, rmse, smape
from utilsforecast.evaluation import evaluate
from utilsforecast.plotting import plot_series

# necessary for unique_id working
os.environ['NIXTLA_ID_AS_COL'] = '1'

C:\Users\cerbe\anaconda3\envs\hw_ts_project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data Preparation

## load basic data

In [2]:
data_paths = {'etag_5n_loc': '../data/cleaned/etag_5n_loc.csv',
              'section_info': '../data/cleaned/section_info.csv',
              'hw5_m04a_df': '../data/cleaned/hw5_m04a.csv',
              'congestion_table': '../data/cleaned/congestion_table.csv',
              'calendar_event': '../data/cleaned/calendar_event.csv',
              'road_build_event': '../data/cleaned/202301_10_road_build_event.xlsx',
              'traffic_accident_data': '../data/cleaned/202301_10_traffic_accident_data.xlsx'
             }

rs = tk.hw_df_resource(data_paths)
rs.load_raw_environment_info()
rs.load_raw_event_info()
rs.generate_mile_location_info()

Complete loading environment and gantry info
Complete loading raw event info
Complete generating mile location info


In [3]:
# 旅行時間的資料
hw5_15watt = pd.read_csv("../data/features/hw5_15watt.csv")
hw5_15watt = hw5_15watt[~hw5_15watt.gf_gt.isin(['03F0150N-03F0140N', 
                                                '03F0201S-03A0041N', 
                                                '03F0201S-03F0217S'])].copy()

In [4]:
hw5_15watt.gf_gt.unique()

array(['05F0001N-03F0150N', '05F0001N-03F0201S', '05F0055N-05F0001N',
       '05F0287N-05F0055N', '05F0309N-05F0287N', '05F0438N-05F0309N',
       '05F0438N-05FR143N', '05F0528N-05F0438N'], dtype=object)

In [5]:
hw5_15watt.shape

(350208, 6)

In [6]:
hw5_15watt.keys()

Index(['gf_gt', 'GantryFrom', 'GantryTo', 'TimeStamp', 'WeightedAvgTravelTime',
       'TotalTraffic'],
      dtype='object')

## data split

這邊要將資料依據門架區間拆分成獨立的df，方便後續根據獨立時間序列進行訓練

In [7]:
df = hw5_15watt[['TimeStamp', 'WeightedAvgTravelTime', 'gf_gt']].copy()#.reset_index()
df.rename(columns={'gf_gt': 'unique_id', 
                   'TimeStamp': 'ds',
                   'WeightedAvgTravelTime': 'y'}, inplace=True)

In [8]:
# data split
f_df = df[df.ds>='2024-01-01 00:00:00'].copy()
p_df = df[df.ds<'2024-01-01 00:00:00'].copy()
f_df.shape, p_df.shape

((69888, 3), (280320, 3))

我們以2023年整年的資料作為訓練集，2024年的會是後續驗證用的部分  

# Ｍodelling

## baseline

Baseline using:
* AutoARIMA
* SeasonalNaive
* MSTL

上述都是可以一定程度上捕捉趨勢的模型  
交通資料在中短期的範圍下是具有季節性存在的  
透過上述三種模型我們可以瞭解使用最簡單的方法下效果可以有多好  
Statsmodel在這塊不需要特別去把序列分拆  
端看當前使用的模型是可以多變量還是單變量來調整輸入  

In [9]:
SEASON_LENGTH=96 # 因為是每15分鐘一筆資料，一天96筆當作season的假設

models = [
    AutoARIMA(),
    SeasonalNaive(season_length=SEASON_LENGTH), # 5分鐘資料換算成1天的話
    MSTL(season_length=SEASON_LENGTH),
]

sf = StatsForecast(
    models=models,
    freq='15min',
    fallback_model = SeasonalNaive(season_length=SEASON_LENGTH),
    n_jobs=-1,
)

## cross validation

因為一年，我們將windows調整成每周，一次輸入兩周的資料  
預測未來一周的成果  

In [10]:
# 端看過去一年來說，一次跳一個月進行驗證，驗證範圍為未來的7天
# 改成一個禮拜跳一次進行驗證
start_time = time.time()
horizon=4*24*7

crossvaldation_df = sf.cross_validation(
    df=p_df,
    h=horizon, # 未來要預測多少步，這邊先假設1天 = 4*24，應該改成一次預測1個禮拜，96*7
    input_size=horizon*2,
    step_size=horizon, # 1次跳1個禮拜
    n_windows=50
)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")

Elapsed time: 51.17 seconds


In [11]:
crossvaldation_df

,unique_id,ds,cutoff,y,AutoARIMA,SeasonalNaive,MSTL
0,05F0001N-03F0150N,2023-01-16 00:00:00,2023-01-15 23:45:00,49.018395,48.877049,49.378951,48.093091
1,05F0001N-03F0150N,2023-01-16 00:15:00,2023-01-15 23:45:00,48.558473,49.573010,48.555556,47.448562
2,05F0001N-03F0150N,2023-01-16 00:30:00,2023-01-15 23:45:00,75.958333,50.107602,48.118138,47.330838
3,05F0001N-03F0150N,2023-01-16 00:45:00,2023-01-15 23:45:00,48.000000,50.558004,48.733656,47.322311
4,05F0001N-03F0150N,2023-01-16 01:00:00,2023-01-15 23:45:00,48.666667,50.904738,48.156111,46.873375
...,...,...,...,...,...,...,...
268795,05F0528N-05F0438N,2023-12-31 22:45:00,2023-12-24 23:45:00,336.021637,349.563554,333.683333,338.231426
268796,05F0528N-05F0438N,2023-12-31 23:00:00,2023-12-24 23:45:00,337.304762,349.563554,320.679506,342.791335
268797,05F0528N-05F0438N,2023-12-31 23:15:00,2023-12-24 23:45:00,337.973434,349.563554,335.102767,336.178585
268798,05F0528N-05F0438N,2023-12-31 23:30:00,2023-12-24 23:45:00,359.978508,349.563554,343.051282,333.614241


以上就是我們挑選的三種baseline模型產出的結果  
我們將其輸出好保留後續參考  

In [12]:
crossvaldation_df.to_csv(f'../outputs/baseline.csv', index=False)

## evaluation

針對每個門架間旅行時間我們可以用以下簡易的函數來判斷其成果  

In [13]:
def evaluate_cross_validation(df, metric_list):
    models = df.drop(columns=['unique_id', 'ds', 'cutoff', 'y']).columns.tolist()
    evals = []
    # Calculate loss for every unique_id and cutoff.    
    for cutoff in df['cutoff'].unique():
        eval_ = evaluate(df[df['cutoff'] == cutoff], metrics=metric_list, models=models)
        evals.append(eval_)
    evals = pd.concat(evals)
    evals = evals.groupby('unique_id').mean(numeric_only=True) # Averages the error metrics for all cutoffs for every combination of model and unique_id
    evals['best_model'] = evals.idxmin(axis=1)
    return evals


In [14]:
evaluation_df_mse = evaluate_cross_validation(crossvaldation_df, [mse])
evaluation_df_mae = evaluate_cross_validation(crossvaldation_df, [mae])
evaluation_df_rmse = evaluate_cross_validation(crossvaldation_df, [rmse])
evaluation_df_smape = evaluate_cross_validation(crossvaldation_df, [smape])

In [15]:
evaluation_df_mse

,AutoARIMA,SeasonalNaive,MSTL,best_model
unique_id,,,,
05F0001N-03F0150N,85.372747,93.813501,9.064918e+01,AutoARIMA
05F0001N-03F0201S,4713.188691,4630.640946,9.785856e+05,SeasonalNaive
05F0055N-05F0001N,4430.850225,10066.319504,2.012572e+05,AutoARIMA
05F0287N-05F0055N,108447.756730,117113.610888,2.034877e+05,AutoARIMA
05F0309N-05F0287N,109258.092428,258666.870796,1.195306e+05,AutoARIMA
05F0438N-05F0309N,89083.831180,257378.970688,5.294957e+06,AutoARIMA
05F0438N-05FR143N,234.031204,311.461353,1.959256e+02,MSTL
05F0528N-05F0438N,639.471247,1233.429062,7.019908e+02,AutoARIMA


In [16]:
evaluation_df_mae

,AutoARIMA,SeasonalNaive,MSTL,best_model
unique_id,,,,
05F0001N-03F0150N,2.946123,2.683655,2.917927,SeasonalNaive
05F0001N-03F0201S,31.947654,23.843967,547.636363,SeasonalNaive
05F0055N-05F0001N,33.351446,48.775680,195.916615,AutoARIMA
05F0287N-05F0055N,228.769277,220.830562,263.657005,SeasonalNaive
05F0309N-05F0287N,198.363102,303.175574,220.041011,AutoARIMA
05F0438N-05F0309N,150.949097,273.208247,1370.550284,AutoARIMA
05F0438N-05FR143N,8.043027,8.643810,7.376302,MSTL
05F0528N-05F0438N,13.506270,18.629390,15.291096,AutoARIMA


In [17]:
evaluation_df_rmse

,AutoARIMA,SeasonalNaive,MSTL,best_model
unique_id,,,,
05F0001N-03F0150N,7.703498,8.128783,8.028405,AutoARIMA
05F0001N-03F0201S,64.813195,63.757025,646.144720,SeasonalNaive
05F0055N-05F0001N,60.970332,94.092144,238.169985,AutoARIMA
05F0287N-05F0055N,311.690557,332.844973,328.645359,AutoARIMA
05F0309N-05F0287N,309.353216,494.039734,307.927923,MSTL
05F0438N-05F0309N,269.175589,475.000663,1593.225491,AutoARIMA
05F0438N-05FR143N,15.134644,16.986104,13.780017,MSTL
05F0528N-05F0438N,24.880836,34.118204,26.097242,AutoARIMA


In [18]:
evaluation_df_smape

,AutoARIMA,SeasonalNaive,MSTL,best_model
unique_id,,,,
05F0001N-03F0150N,0.026173,0.023014,0.025181,SeasonalNaive
05F0001N-03F0201S,0.059862,0.041150,0.348561,SeasonalNaive
05F0055N-05F0001N,0.060208,0.072614,0.175746,AutoARIMA
05F0287N-05F0055N,0.094618,0.082927,0.095612,SeasonalNaive
05F0309N-05F0287N,0.372934,0.302044,0.396003,SeasonalNaive
05F0438N-05F0309N,0.102619,0.143851,0.402312,AutoARIMA
05F0438N-05FR143N,0.042634,0.046742,0.040351,MSTL
05F0528N-05F0438N,0.019244,0.026499,0.021895,AutoARIMA
